# Full train and testing

In [ ]:
from time import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from data import input_data

In [ ]:
from utils import *
from data import input_data
from model import *

In [ ]:
criterion_d = nn.CrossEntropyLoss()
input_train = input_data(root_dir = "/home/hemant/net/easy_net/data/train/", type = "valid")
train_dl =  DataLoader(input_train, batch_size=32,shuffle=True, num_workers=4)
input_valid = input_data(root_dir = "/home/hemant/net/easy_net/data/test/", type = "valid")
valid_dl =  DataLoader(input_valid, batch_size=64,shuffle=False, num_workers=4)

In [ ]:
model = nn.Sequential(  feature_b(),
                        feature_r(),
                        decision(out_classes = input_train[0][3]))

In [ ]:
rank = "2"
optimizer = optim.Adam(model.parameters(), lr=0.001)
device = torch.device("cuda:"+rank)
model.to(device)
model.load_state_dict(torch.load("../weights/full.pth",map_location="cuda:"+str(rank)),strict=True)
print("")

# Training

In [ ]:
accuracy = 0
for j in range(20):
    print("start of epoch: ", j+1)
    #Training
    model.train()
    running_loss = 0
    start = time()
    for i, data in enumerate(train_dl, 0):

        input, target, img_name, number_of_class = data
        input, target = (input.type(torch.float32)).to(device), target.to(device)

        out = model(input)

        loss = criterion_d(out, target)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()


        optimizer.zero_grad()
        
        # print every 25 mini-batches
        if i % 25 == 24:
            print('[%d, %5d] loss: %.3f' %(j + 1, i + 1, running_loss))
            running_loss = 0
    end = time()
    print(" It took : ", (end - start)/60, " mins for the last training epoch")
    
    running_loss, acc, num, length = 0, 0, 0, 0
    with torch.no_grad():
        start = time()
        for i, data in enumerate(valid_dl, 0):
            model.eval()

            input, target, img_name, number_of_class = data
            input, target = (input.type(torch.float32)).to(device), target.to(device)

            out = model(input)

            loss = criterion_d(out, target)

            out , predicted = torch.max(out, 1)
            for k in range(len(target)):
                if target[k] == predicted[k].item():
                    num = num + 1
            length = length + len(target)
        acc = (num/length)*100
        end = time()
        if acc>accuracy:
            torch.save(model.state_dict(),"../weights/full.pth")
            accuracy = acc
        print("accuracy and val loss is : ",acc,",",running_loss/(i+1), " --AND-- ", " It took : ", (end - start), " seconds ")

# Testing

In [ ]:
running_loss, acc, num, length = 0, 0, 0, 0
with torch.no_grad():
    start = time()
    for i, data in enumerate(valid_dl, 0):
        model.eval()

        input, target, img_name, number_of_class = data
        input, target = (input.type(torch.float32)).to(device), target.to(device)

        out = model(input)

        loss = criterion_d(out, target)

        out , predicted = torch.max(out, 1)
        for k in range(len(target)):
            if target[k] == predicted[k].item():
                num = num + 1
        length = length + len(target)
    acc = (num/length)*100
    end = time()
    print("accuracy and val loss is : ",acc,",",running_loss/(i+1), " --AND-- ", " It took : ", (end - start), " seconds ")